In [ ]:
import numpy as np
import pandas as pd
import os, math, sys
import time, datetime
import glob, itertools
import argparse, random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torch.autograd import Variable
from torchvision.models import vgg19
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image, make_grid

import plotly
from scipy import signal
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

random.seed(42)
import warnings
warnings.filterwarnings("ignore")

### Settings

In [ ]:
# path to pre-trained models
# pretrained_model_path = "../input/cyclegan-translating-horses-zebras-pytorch/saved_models"

# epoch to start training from
epoch_start = 0
# number of epochs of training
n_epochs = 10

# name of the dataset
# dataset_path = "../input/horse2zebra-dataset"

# size of the batches"
batch_size = 4
# adam: learning rate
lr = 0.02
# adam: decay of first order momentum of gradient
b1 = 0.5
# adam: decay of first order momentum of gradient
b2 = 0.999
# epoch from which to start lr decay
decay_epoch = 1
# number of cpu threads to use during batch generation
n_workers = 16
# size of image height
img_height = 256
# size of image width
img_width = 256
# number of image channels
channels = 1
# interval between saving generator outputs
sample_interval = 100
# interval between saving model checkpoints
checkpoint_interval = -1
# number of residual blocks in generator
n_residual_blocks = 9
# cycle loss weight
lambda_cyc = 10.0
# identity loss weight
lambda_id = 5.0
# Development / Debug Mode
debug_mode = False

# Create images and checkpoint directories
os.makedirs("images", exist_ok=True)
os.makedirs("saved_models", exist_ok=True)

### Define Utilities

In [ ]:

class ReplayBuffer:
    def __init__(self, max_size=50):
        assert max_size > 0
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)
            if len(self.data) < self.max_size:
                self.data.append(element)
                to_return.append(element)
            else:
                if random.uniform(0, 1) > 0.5:
                    i = random.randint(0, self.max_size - 1)
                    to_return.append(self.data[i].clone())
                    self.data[i] = element
                else:
                    to_return.append(element)
        return Variable(torch.cat(to_return))


### Define Dataset Class

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned

        self.files_A = sorted(glob.glob(os.path.join(root, f"{mode}A") + "/*.*"))
        self.files_B = sorted(glob.glob(os.path.join(root, f"{mode}B") + "/*.*"))
        if debug_mode:
            self.files_A = self.files_A[:100]
            self.files_B = self.files_B[:100]

    def __getitem__(self, index):
        image_A = Image.open(self.files_A[index % len(self.files_A)])

        if self.unaligned:
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

        # Convert grayscale images to rgb
#         if image_A.mode != "RGB":
#             image_A = to_rgb(image_A)
#         if image_B.mode != "RGB":
#             image_B = to_rgb(image_B)

        item_A = self.transform(image_A)
        item_B = self.transform(image_B)
        return {"A": item_A, "B": item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))

### Get Train/Test Dataloaders

In [ ]:
# pip install split-folders

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
path="/kaggle/working/"
os.makedirs(os.path.join(path,'data'),exist_ok=True)

path='/kaggle/working/data'
os.makedirs(os.path.join(path,'trainA'),exist_ok=True)
os.makedirs(os.path.join(path,'trainB'),exist_ok=True)


def organize_folders(root_folder, dest_folder):
    for split_folder in ['test', 'train', 'val']:
        split_path = os.path.join(root_folder, split_folder)

        for subfolder in os.listdir(split_path):
            subfolder_path = os.path.join(split_path, subfolder)
            dest_path = os.path.join(dest_folder, subfolder)

            for image_file in os.listdir(subfolder_path):
                source_path = os.path.join(subfolder_path, image_file)

                if 'bg' in image_file:
                    destination_folder = 'trainA'
#                     continue
                elif 'cl' in image_file:
#                     destination_folder = 'trainA'
                    continue
                elif 'nm' in image_file:
                    destination_folder = 'trainB'
#                 print(image_file)
                # Constructing the destination path using os.path.join
                destination_path = os.path.join(dest_folder, destination_folder, os.path.basename(image_file))
                shutil.copy(source_path, destination_path)

# Example usage
root_folder = "/kaggle/input/casiab-identification/DATASETidentification"
dest_folder = "/kaggle/working/data/"
organize_folders(root_folder, dest_folder)


# splitfolders.ratio(input_folder, output=output_folder, seed=1337, ratio=ratio, group_prefix=None)


In [ ]:
import shutil
import random


os.makedirs("/kaggle/working/data/testA", exist_ok=True)
os.makedirs("/kaggle/working/data/testB", exist_ok=True)

images=os.listdir("/kaggle/working/data/trainA")

img=random.sample(images,815)
# img=random.sample(images,1631)

for image in img:
    src = os.path.join("/kaggle/working/data/trainA", image)
    dst = os.path.join("/kaggle/working/data/testA", image)
    shutil.move(src, dst)


images=os.listdir("/kaggle/working/data/trainB")
img=random.sample(images,2446)
for image in img:
    src = os.path.join("/kaggle/working/data/trainB", image)
    dst = os.path.join("/kaggle/working/data/testB", image)
    shutil.move(src, dst)


     

In [ ]:
images=os.listdir("/kaggle/working/data/trainA")
len(images)


In [ ]:
# Image transformations
transforms_ = [
    transforms.Resize(int(img_height * 1.12), Image.BICUBIC),
    transforms.RandomCrop((img_height, img_width)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),

]



# Training data loader
train_dataloader = DataLoader(
    ImageDataset("/kaggle/working/data/", transforms_=transforms_, unaligned=True),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_workers,
)
# Test data loader
test_dataloader = DataLoader(
    ImageDataset("/kaggle/working/data/", transforms_=transforms_, unaligned=True, mode="test"),
    batch_size=1,
    shuffle=True,
    num_workers=1,
)

In [ ]:
image = next(iter(train_dataloader))
print(image.get('A').shape,image.get('B').shape)

### Define Model Classes

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, "bias") and m.bias is not None:
            torch.nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
        
        
# class SelfAttention(nn.Module):
#     def __init__(self, in_channels):
#         super(SelfAttention, self).__init__()
#         self.query_conv = nn.Conv2d(in_channels, in_channels // 8, 1)
#         self.key_conv = nn.Conv2d(in_channels, in_channels // 8, 1)
#         self.value_conv = nn.Conv2d(in_channels, in_channels, 1)
#         self.gamma = nn.Parameter(torch.zeros(1))

#     def forward(self, x):
#         batch_size, channels, height, width = x.size()
#         proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
#         proj_key = self.key_conv(x).view(batch_size, -1, width * height)
#         energy = torch.bmm(proj_query, proj_key)
#         attention = F.softmax(energy, dim=-1)
#         proj_value = self.value_conv(x).view(batch_size, -1, width * height)
#         out = torch.bmm(proj_value, attention.permute(0, 2, 1))
#         out = out.view(batch_size, channels, height, width)
#         out = self.gamma * out + x
#         return out

    
# class ResidualBlock(nn.Module):
#     def __init__(self, in_features):
#         super(ResidualBlock, self).__init__()
#         self.block = nn.Sequential(
#             nn.ReflectionPad2d(1),
#             nn.Conv2d(in_features, in_features, 3),
#             nn.BatchNorm2d(in_features),
#             nn.ReLU(inplace=True),
#             SelfAttention(in_features),  # Add self-attention mechanism
#             nn.ReflectionPad2d(1),
#             nn.Conv2d(in_features, in_features, 3),
#             nn.BatchNorm2d(in_features),
#         )

#     def forward(self, x):
#         return x + self.block(x)

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.block = nn.Sequential(
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.BatchNorm2d(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            nn.BatchNorm2d(in_features),
        )

    def forward(self, x):
        return x + self.block(x)


class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks):
        super(GeneratorResNet, self).__init__()

        channels = input_shape[0]

        # Initial convolution block
        out_features = 64
        model = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(channels, out_features, 7),
            nn.BatchNorm2d(out_features),
            nn.ReLU(inplace=True),
        ]
        in_features = out_features

        # Downsampling
        for _ in range(2):
            out_features *= 2
            model += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                nn.BatchNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Residual blocks
        for _ in range(num_residual_blocks):
            model += [ResidualBlock(out_features)]

        # Upsampling
        for _ in range(2):
            out_features //= 2
            model += [
                nn.Upsample(scale_factor=2),
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),
                nn.BatchNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features

        # Output layer
        model += [nn.ReflectionPad2d(3), nn.Conv2d(out_features, channels, 7), nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        return self.model(x)


class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        channels, height, width = input_shape

        # Calculate output shape of image discriminator (PatchGAN)
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)

        def discriminator_block(in_filters, out_filters, normalize=True):
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img):
        return self.model(img)

In [ ]:
g=GeneratorResNet((1,256,256),9)
d=Discriminator((1,256,256))
dummy = torch.randn(1, 1, 256, 256)
out_gen=g(dummy)
out_disc=d(dummy)
print(out_gen.shape,out_disc.shape)

### Train CycleGAN

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
def displayImage(x):
    x=x[0].cpu()
    image_bag = (x.permute(1,2,0).detach().numpy() * 255).astype('uint8')
    plt.imshow(image_bag,cmap="gray")
    plt.show()

In [ ]:
# Losses

def wgan_loss(real_output, fake_output):
    return -torch.mean(real_output) + torch.mean(fake_output)


criterion_GAN = torch.nn.MSELoss()
criterion_cycle = torch.nn.L1Loss()
criterion_identity = torch.nn.L1Loss()

cuda = torch.cuda.is_available()

input_shape = (channels, img_height, img_width)
# Initialize generator and discriminator
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
G_BA = GeneratorResNet(input_shape, n_residual_blocks)
D_A = Discriminator(input_shape)
D_B = Discriminator(input_shape)

if cuda:
    G_AB = G_AB.cuda()
    G_BA = G_BA.cuda()
    D_A = D_A.cuda()
    D_B = D_B.cuda()
    criterion_GAN.cuda()
    criterion_cycle.cuda()
    criterion_identity.cuda()

if epoch_start != 0:
    # Load pretrained models
    G_AB.load_state_dict(torch.load(f"{pretrained_model_path}/G_AB.pth"))
    G_BA.load_state_dict(torch.load(f"{pretrained_model_path}/G_BA.pth"))
    D_A.load_state_dict(torch.load(f"{pretrained_model_path}/D_A.pth"))
    D_B.load_state_dict(torch.load(f"{pretrained_model_path}/D_B.pth"))
else:
    # Initialize weights
    G_AB.apply(weights_init_normal)
    G_BA.apply(weights_init_normal)
    D_A.apply(weights_init_normal)
    D_B.apply(weights_init_normal)

# Optimizers
optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr, betas=(b1, b2)
)

# optimizer_G_AB = torch.optim.Adam(G_AB.parameters(), lr=lr, betas=(b1, b2))
# optimizer_G_BA = torch.optim.Adam(G_BA.parameters(), lr=lr, betas=(b1, b2))

optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(b1, b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

# Buffers of previously generated samples
fake_A_buffer = ReplayBuffer()
fake_B_buffer = ReplayBuffer()

train_counter = []
train_losses_gen, train_losses_id, train_losses_gan, train_losses_cyc = [], [], [], []
train_losses_disc, train_losses_disc_a, train_losses_disc_b = [], [], []

test_counter = [2*idx*len(train_dataloader.dataset) for idx in range(epoch_start+1, n_epochs+1)]
test_losses_gen, test_losses_disc = [], []


In [ ]:
import torch
torch.cuda.empty_cache()
# Check available memory on CUDA device
cuda_memory_stats = torch.cuda.memory_stats()
free_memory_bytes = cuda_memory_stats['allocated_bytes.all.peak'] - cuda_memory_stats['allocated_bytes.all.current']
free_memory_mb = free_memory_bytes / 1024 / 1024
print("Free CUDA memory:", free_memory_mb, "MB")


In [ ]:
from tqdm import tqdm_notebook as tqdm

torch.cuda.empty_cache()
for epoch in range(epoch_start, n_epochs):
    #### Training
    loss_gen = loss_id = loss_gan = loss_cyc = 0.0
    loss_disc = loss_disc_a = loss_disc_b = 0.0
    tqdm_bar = tqdm(train_dataloader, desc=f'Training Epoch {epoch} ', total=int(len(train_dataloader)))
    for batch_idx, batch in enumerate(tqdm_bar):
        # Set model input
        real_A = Variable(batch["A"].type(Tensor))
        real_B = Variable(batch["B"].type(Tensor))
        # Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False)
        fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))), requires_grad=False)
        

        ### Train Generators
        G_AB.train()
        G_BA.train()
        # Identity loss
        loss_id_A = criterion_identity(G_BA(real_A), real_A)
        loss_id_B = criterion_identity(G_AB(real_B), real_B)
        loss_identity = (loss_id_A + loss_id_B) / 2
        # GAN loss
        fake_B = G_AB(real_A)
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
        fake_A = G_BA(real_B)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)
        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2
        # Cycle loss
        recov_A = G_BA(fake_B)
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        recov_B = G_AB(fake_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)
        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2

        # Total loss for each generator
#         loss_G_AB = lambda_id * loss_id_A + loss_GAN_AB + lambda_cyc * loss_cycle_A
#         loss_G_BA = lambda_id * loss_id_B + loss_GAN_BA + lambda_cyc * loss_cycle_B
#         optimizer_G_AB.zero_grad()
#         optimizer_G_BA.zero_grad()
#         loss_G_AB.backward()
#         loss_G_BA.backward()
#         optimizer_G_AB.step()
#         optimizer_G_BA.step()
        
        # Total loss
        loss_G = lambda_id * loss_identity + loss_GAN + lambda_cyc * loss_cycle
        loss_G.backward()
        optimizer_G.step()
        

        ### Train Discriminator-A
        D_A.train()
        optimizer_D_A.zero_grad()
        
        # Real loss
        loss_real = criterion_GAN(D_A(real_A), valid)
#         loss_real = wgan_loss(D_A(real_A), valid)


        # Fake loss (on batch of previously generated samples)
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)
        
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
#         loss_fake = wgan_loss(D_A(fake_A_.detach()), fake)

        # Total loss
        loss_D_A = (loss_real + loss_fake) / 2
        loss_D_A.backward()
        optimizer_D_A.step()

        ### Train Discriminator-B
        D_B.train()
        optimizer_D_B.zero_grad()
        # Real loss
        loss_real = criterion_GAN(D_B(real_B), valid)
#         loss_real = wgan_loss(D_B(real_B), valid)

        # Fake loss (on batch of previously generated samples)
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
#         loss_fake = wgan_loss(D_B(fake_B_.detach()), fake)

        # Total loss
        loss_D_B = (loss_real + loss_fake) / 2
        loss_D_B.backward()
        optimizer_D_B.step()
        loss_D = (loss_D_A + loss_D_B) / 2

        ### Log Progress
        loss_gen += loss_G.item(); loss_id += loss_identity.item(); loss_gan += loss_GAN.item(); loss_cyc += loss_cycle.item()
        loss_disc += loss_D.item(); loss_disc_a += loss_D_A.item(); loss_disc_b += loss_D_B.item()
        train_counter.append(2*(batch_idx*batch_size + real_A.size(0) + epoch*len(train_dataloader.dataset)))
        train_losses_gen.append(loss_G.item()); train_losses_id.append(loss_identity.item()); train_losses_gan.append(loss_GAN.item()); train_losses_cyc.append(loss_cycle.item())
        train_losses_disc.append(loss_D.item()); train_losses_disc_a.append(loss_D_A.item()); train_losses_disc_b.append(loss_D_B.item())
        tqdm_bar.set_postfix(Gen_loss=loss_gen/(batch_idx+1), identity=loss_id/(batch_idx+1), adv=loss_gan/(batch_idx+1), cycle=loss_cyc/(batch_idx+1),
                            Disc_loss=loss_disc/(batch_idx+1), disc_a=loss_disc_a/(batch_idx+1), disc_b=loss_disc_b/(batch_idx+1))

    real_A = make_grid(real_A, nrow=1, normalize=True)
    real_B = make_grid(real_B, nrow=1, normalize=True)
    fake_A = make_grid(fake_A, nrow=1, normalize=True)
    fake_B = make_grid(fake_B, nrow=1, normalize=True)
    # Arange images along y-axis
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), -1)

    image_grid_np = image_grid.permute(1, 2, 0).cpu().detach().numpy()

    # Display the image
    plt.imshow(image_grid_np)
    plt.axis('off')
    plt.show()
    torch.save(G_AB.state_dict(), "saved_models/G_AB.pth")

In [ ]:
### Testing
import matplotlib.pyplot as plt

loss_gen = loss_id = loss_gan = loss_cyc = 0.0
loss_disc = loss_disc_a = loss_disc_b = 0.0
tqdm_bar = tqdm(test_dataloader, desc=f'Testing epoch', total=int(len(test_dataloader)))
for batch_idx, batch in enumerate(tqdm_bar):
    # Set model input
    real_A = Variable(batch["A"].type(Tensor))
    real_B = Variable(batch["B"].type(Tensor))
    # Adversarial ground truths
    valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False)
    fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))), requires_grad=False)

    ### Test Generators
    G_AB.eval()
    G_BA.eval()
    # Identity loss
    loss_id_A = criterion_identity(G_BA(real_A), real_A)
    loss_id_B = criterion_identity(G_AB(real_B), real_B)
    loss_identity = (loss_id_A + loss_id_B) / 2
    # GAN loss
    fake_B = G_AB(real_A)
    loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)
    fake_A = G_BA(real_B)
    loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)
    loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2
    # Cycle loss
    recov_A = G_BA(fake_B)
    loss_cycle_A = criterion_cycle(recov_A, real_A)
    recov_B = G_AB(fake_A)
    loss_cycle_B = criterion_cycle(recov_B, real_B)
    loss_cycle = (loss_cycle_A + loss_cycle_B) / 2
    # Total loss
    loss_G = loss_GAN + lambda_cyc * loss_cycle + lambda_id * loss_identity

    ### Test Discriminator-A
    D_A.eval()
    # Real loss
    loss_real = criterion_GAN(D_A(real_A), valid)
    # Fake loss (on batch of previously generated samples)
    fake_A_ = fake_A_buffer.push_and_pop(fake_A)
    loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)
    # Total loss
    loss_D_A = (loss_real + loss_fake) / 2

    ### Test Discriminator-B
    D_B.eval()
    # Real loss
    loss_real = criterion_GAN(D_B(real_B), valid)
    # Fake loss (on batch of previously generated samples)
    fake_B_ = fake_B_buffer.push_and_pop(fake_B)
    loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)
    # Total loss
    loss_D_B = (loss_real + loss_fake) / 2
    loss_D = (loss_D_A + loss_D_B) / 2

    ### Log Progress
    loss_gen += loss_G.item(); loss_id += loss_identity.item(); loss_gan += loss_GAN.item(); loss_cyc += loss_cycle.item()
    loss_disc += loss_D.item(); loss_disc_a += loss_D_A.item(); loss_disc_b += loss_D_B.item()
    tqdm_bar.set_postfix(Gen_loss=loss_gen/(batch_idx+1), identity=loss_id/(batch_idx+1), adv=loss_gan/(batch_idx+1), cycle=loss_cyc/(batch_idx+1),
                        Disc_loss=loss_disc/(batch_idx+1), disc_a=loss_disc_a/(batch_idx+1), disc_b=loss_disc_b/(batch_idx+1))


    # If at sample interval save image
    if random.uniform(0,1)<0.4:
        # Arrange images along x-axis
        real_A = make_grid(real_A, nrow=1, normalize=True)
        real_B = make_grid(real_B, nrow=1, normalize=True)
        fake_A = make_grid(fake_A, nrow=1, normalize=True)
        fake_B = make_grid(fake_B, nrow=1, normalize=True)
        # Arange images along y-axis
        image_grid = torch.cat((real_A, fake_B, real_B, fake_A), -1)
        
        image_grid_np = image_grid.permute(1, 2, 0).cpu().detach().numpy()
    
        # Display the image
        plt.imshow(image_grid_np)
        plt.axis('off')
        plt.show()

test_losses_gen.append(loss_gen/len(test_dataloader))
test_losses_disc.append(loss_disc/len(test_dataloader))

    # Save model checkpoints
if np.argmin(test_losses_gen) == len(test_losses_gen)-1:
    # Save model checkpoints
    torch.save(G_AB.state_dict(), "saved_models/G_AB.pth")
    torch.save(G_BA.state_dict(), "saved_models/G_BA.pth")
    torch.save(D_A.state_dict(), "saved_models/D_A.pth")
    torch.save(D_B.state_dict(), "saved_models/D_B.pth")

In [ ]:
#load weights to generator and discriminator

# G_AB=torch.load('/kaggle/input/weight-files/G_AB.pth')
# G_BA=torch.load('/kaggle/input/weight-files/G_BA.pth')
# D_A=torch.load('/kaggle/input/weight-files/D_A.pth')
# D_B=torch.load('/kaggle/input/weight-files/D_B.pth')


model_path = '/kaggle/input/weight-files/G_AB (3).pth'

# model_path='/kaggle/working/saved_models/G_AB.pth'
model_state = torch.load(model_path, map_location=torch.device('cpu'))
G_AB = GeneratorResNet(input_shape, n_residual_blocks)
G_AB.load_state_dict(model_state)  
G_AB.eval() 

# model_path = '/kaggle/input/weight-files/G_BA.pth'
# model_state = torch.load(model_path, map_location=torch.device('cpu'))
# G_BA = GeneratorResNet(input_shape, n_residual_blocks)
# G_BA.load_state_dict(model_state)  
# G_BA.eval() 

In [ ]:
import matplotlib.pyplot as plt

G_AB.cuda()
G_AB.eval()

tqdm_bar = tqdm(test_dataloader, desc=f'Testing epoch', total=int(len(test_dataloader)))
for batch_idx, batch in enumerate(tqdm_bar):
    # Set model input
    real_A = Variable(batch["A"].type(Tensor)).cuda()
    fake_B=G_AB(real_A).cuda()

    real_A = make_grid(real_A, nrow=1, normalize=True)
    fake_B = make_grid(fake_B, nrow=1, normalize=True)

    image_grid = torch.cat((real_A, fake_B), -1)
    
    image_grid_np = image_grid.permute(1, 2, 0).cpu().detach().numpy()
    
    plt.imshow(image_grid_np)
    plt.axis('off')
    plt.show()
    if(batch_idx==5):
        break

### CNN

In [ ]:
# import torch
# import torch.nn as nn
# from torchvision import transforms
# from PIL import Image
# import matplotlib.pyplot as plt

# # Define your model architecture
# class CNNModel(nn.Module):
#     def __init__(self):
#         super(CNNModel, self).__init__()
#         # Convolutional layers
#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
#         self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
#         self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        
#         # Maxpool layers
#         self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
#         # Fully connected layer
#         self.fc = nn.Linear(512 * 15 * 15, 124)  # Adjust the input size based on the output size of the last convolutional layer
        
#         # Xavier Initialization
#         self.initialize_weights()

#     def forward(self, x):
#         # Convolutional layers with ReLU activation and maxpooling
#         x = F.relu(self.conv1(x))
#         x = self.maxpool(x)
#         x = F.relu(self.conv2(x))
#         x = self.maxpool(x)
#         x = F.relu(self.conv3(x))
#         x = self.maxpool(x)
#         x = F.relu(self.conv4(x))
#         x = self.maxpool(x)
        
#         # Flatten the output for fully connected layers
#         x = x.view(-1, 512 * 15 * 15)
        
#         # Fully connected layer
#         x = self.fc(x)
        
#         return x
    
#     def initialize_weights(self):
#         # Xavier Initialization for convolutional layers
#         nn.init.xavier_uniform_(self.conv1.weight)
#         nn.init.xavier_uniform_(self.conv2.weight)
#         nn.init.xavier_uniform_(self.conv3.weight)
#         nn.init.xavier_uniform_(self.conv4.weight)
        
#         # Xavier Initialization for fully connected layer
#         nn.init.xavier_uniform_(self.fc.weight)

# # Load the trained model
# model_path = '/kaggle/input/weight-files/cnn-0.5.pth'
# model_state = torch.load(model_path, map_location=torch.device('cpu'))
# model = CNNModel()
# model.load_state_dict(model_state)

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SelfAttention, self).__init__()
        self.query = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.key = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.value = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        energy = torch.matmul(query, key.transpose(2, 3))  # Calculate energy scores
        attention_weights = self.softmax(energy)  # Apply softmax to get attention weights
        attended_values = torch.matmul(attention_weights, value)  # Apply attention weights to values

        return attended_values


class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        
        # Maxpool layers
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.attention = SelfAttention(in_channels=512, out_channels=512)
        
        # Fully connected layer
        self.fc = nn.Linear(512 * 15 * 15, 124)  # Adjust the input size based on the output size of the last convolutional layer
        
        # Xavier Initialization
        self.initialize_weights()

    def forward(self, x):
        # Convolutional layers with ReLU activation and maxpooling
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)
        x = F.relu(self.conv3(x))
        x = self.maxpool(x)
        x = F.relu(self.conv4(x))
        x = self.maxpool(x)
        
        x = self.attention(x)
        
        # Flatten the output for fully connected layers
        x = x.view(-1, 512 * 15 * 15)
        
        # Fully connected layer
        x = self.fc(x)
        
        return x
    
    def initialize_weights(self):
        # Xavier Initialization for convolutional layers
        nn.init.xavier_uniform_(self.conv1.weight)
        nn.init.xavier_uniform_(self.conv2.weight)
        nn.init.xavier_uniform_(self.conv3.weight)
        nn.init.xavier_uniform_(self.conv4.weight)
        
        # Xavier Initialization for fully connected layer
        nn.init.xavier_uniform_(self.fc.weight)

# Load the trained model
model_path = '/kaggle/input/weight-files/cnn_attention_1.pth'
model_state = torch.load(model_path, map_location=torch.device('cpu'))  # Load model parameters
model = CNNModel()  # Initialize your model
model.load_state_dict(model_state, strict=False)   # Load model parameters (allowing for mismatch)


In [ ]:
cuda = torch.cuda.is_available()

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision import transforms


arr=[1,1,10,100,101,102,103,104,105,106,107,108,109,
     11,110,111,112,113,114,115,116,117,118,119,
     12,121,122,123,124,
    13,14,15,16,17,18,19,
    2,20,21,22,23,24,25,26,27,28,29,
    3,30,31,32,33,34,35,36,37,38,39,
    4,40,41,42,43,44,45,46,47,48,49,
    5,50,51,52,53,54,55,56,57,58,59,
    6,60,61,62,63,64,65,66,67,68,69,
    7,70,71,72,73,74,75,76,77,78,79,
    8,80,81,82,83,84,85,86,87,88,89,
    9,90,91,92,93,94,95,96,97,98,99,]

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((240, 240)),               
    transforms.ToTensor(),                       
    transforms.Normalize(mean=[0.5], std=[0.5])
])


images=os.listdir('/kaggle/working/data/testA')

total_correct=0
total_images=0

G_AB.cuda()
G_AB.eval()
    
truth=[]
predicted=[]
    
for img in images:
    
    image_path = '/kaggle/working/data/testA/'+img
    input_image = Image.open(image_path)    
    input_tensor = transform(input_image)
    input_tensor = input_tensor.unsqueeze(0)
    if cuda:
        input_tensor = input_tensor.cuda()
        model.cuda()
    
    model.eval()
    
    with torch.no_grad():
        generated_image = G_AB(input_tensor)
    
#     real_A = make_grid(input_tensor, nrow=1, normalize=True)
#     fake_B = make_grid(generated_image, nrow=1, normalize=True)
    

#     image_grid = torch.cat((real_A, fake_B), -1)
    
#     image_grid_np = image_grid.permute(1, 2, 0).cpu().detach().numpy()
    
#     plt.imshow(image_grid_np)
#     plt.axis('off')
#     plt.show()

    with torch.no_grad():
        output = model(generated_image)
#         output=model(input_tensor)


    _, predicted_class = torch.max(output, 1)
    predicted_class = predicted_class.item()
    
    ground_truth = int(img[:3])
    
    truth.append(ground_truth)
    predicted.append(predicted_class+1)
    
    

#     print(img[:3], " - ",predicted_class+1)

#     if ground_truth == predicted_class+1:
#         total_correct += 1
    
#     total_images += 1

# precision = total_correct / total_images
# print("Precision:", precision)

# print(truth,predicted)

# Compute accuracy
accuracy = accuracy_score(truth, predicted)

# Compute precision
precision = precision_score(truth, predicted, average='weighted')

# Compute recall
recall = recall_score(truth, predicted, average='weighted')

# Compute F1 score
f1 = f1_score(truth, predicted, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_counter, y=train_losses_gen, mode='lines', name='Train Gen Loss (Loss_G)'))
fig.add_trace(go.Scatter(x=train_counter, y=train_losses_id, mode='lines', name='Train Gen Identity Loss'))
fig.add_trace(go.Scatter(x=train_counter, y=train_losses_gan, mode='lines', name='Train Gen GAN Loss'))
fig.add_trace(go.Scatter(x=train_counter, y=train_losses_cyc, mode='lines', name='Train Gen Cyclic Loss'))
fig.add_trace(go.Scatter(x=test_counter, y=test_losses_gen, marker_symbol='star-diamond', 
                         marker_color='orange', marker_line_width=1, marker_size=9, mode='markers', name='Test Gen Loss (Loss_G)'))
fig.update_layout(
    width=1000,
    height=500,
    title="Train vs. Test Generator Loss",
    xaxis_title="Number of training examples seen (A+B)",
    yaxis_title="Generator Losses"),
plotly.offline.plot(fig, filename = 'plotly_gen_losses.html')
fig.show()

In [ ]:
import shutil
import os

# Specify the path to the working directory
working_directory = "/kaggle/working/saved_models"

# Iterate over the files in the working directory and remove them
for filename in os.listdir(working_directory):
    file_path = os.path.join(working_directory, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print(f"Failed to delete {file_path}: {e}")
